In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [2]:
df = pd.read_csv('sat.trn',header=None,sep = '\s')

In [3]:
X = df.iloc[:,:-1].to_numpy()
y = df.iloc[:,-1].to_numpy()

# 7-> 6 and 1-6 -> 0:5
y[y==7] = 6
for i in range(6):
    y[y == (i + 1)] = i

In [4]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [5]:
def eval_genomes(genomes, config):
    for genome_id, genome in genomes:
        net = neat.nn.FeedForwardNetwork.create(genome, config)
        
        outputs = []
        for xi in x_inputs:
            output = net.activate(xi)
            outputs.append(output)
        
        
        px_outputs = softmax(np.array(outputs).reshape(samplesize, number_of_classification), axis=1)
        # the index of maximum in each line
        pred_outputs = np.argmax(px_outputs, axis = 1)
        # replace index to real number, from higher number to lower number!!
        for i in range(number_of_classification):
            index = number_of_classification - i - 1
            pred_outputs[pred_outputs==index] = numbers[index]
        
        real_outputs = np.array(x_outputs).reshape(samplesize,)
        
        acc = np.sum(pred_outputs == real_outputs)/samplesize

        genome.fitness = acc

In [7]:
numbers = [*range(6)] # number 2 > number 1
number_of_classification = 6


list_y = y.tolist()
digits_indexes = []
for digit in numbers:
    li = [i for i in range(len(list_y)) if list_y[i] == digit]
    digits_indexes.extend(li)

samplesize = X[digits_indexes].shape[0]
x_inputs =[tuple(c) for c in X[digits_indexes].tolist()]
x_outputs = [tuple(c) for c in y[digits_indexes].reshape(samplesize,1).tolist()]

In [12]:
def run(config_file):
    # Load configuration.
    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_file,
    )

    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)

    # add a stdout reporter to show progress in the terminal
    reporter = neat.StdOutReporter(False)
    p.add_reporter(reporter)
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    #checkpointer = neat.Checkpointer(100)
    #p.add_reporter(checkpointer)
    # Run for up to 300 generations.
    winner = p.run(eval_genomes, 300)

    return [stats, winner]


samplesize = len(x_outputs)

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-satellite6")
[stats, winner] = run(config_path)

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)
winner_net = neat.nn.FeedForwardNetwork.create(winner, config)


 ****** Running generation 0 ****** 

Population's average fitness: 0.18162 stdev: 0.07443
Best fitness: 0.33168 - size: (6, 22) - species 11 - id 182
Average adjusted fitness: 0.188
Mean genetic distance 2.801, standard deviation 0.339
Population of 201 members in 30 species
Total extinctions: 0
Generation time: 23.078 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.19592 stdev: 0.07314
Best fitness: 0.42931 - size: (7, 24) - species 10 - id 323
Average adjusted fitness: 0.178
Mean genetic distance 2.879, standard deviation 0.380
Population of 198 members in 32 species
Total extinctions: 0
Generation time: 19.890 sec (21.484 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.20737 stdev: 0.07153
Best fitness: 0.42931 - size: (7, 24) - species 10 - id 323
Average adjusted fitness: 0.192
Mean genetic distance 2.893, standard deviation 0.387
Population of 199 members in 35 species
Total extinctions: 0
Generation time: 20.113 sec (2

Population's average fitness: 0.24283 stdev: 0.10514
Best fitness: 0.47148 - size: (8, 25) - species 8 - id 721
Average adjusted fitness: 0.217
Mean genetic distance 3.143, standard deviation 0.473
Population of 208 members in 25 species
Total extinctions: 0
Generation time: 23.477 sec (23.336 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.23936 stdev: 0.09897
Best fitness: 0.47148 - size: (8, 25) - species 8 - id 721
Average adjusted fitness: 0.224
Mean genetic distance 3.136, standard deviation 0.463
Population of 209 members in 25 species
Total extinctions: 0
Generation time: 25.047 sec (23.546 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.24239 stdev: 0.09780
Best fitness: 0.47148 - size: (8, 25) - species 8 - id 721
Average adjusted fitness: 0.209
Mean genetic distance 3.137, standard deviation 0.467
Population of 205 members in 24 species
Total extinctions: 0
Generation time: 25.504 sec (23.807 average)

 ******

Population's average fitness: 0.24770 stdev: 0.10837
Best fitness: 0.53551 - size: (11, 28) - species 8 - id 5593
Average adjusted fitness: 0.212
Mean genetic distance 3.281, standard deviation 0.536
Population of 197 members in 20 species
Total extinctions: 0
Generation time: 27.982 sec (27.580 average)

 ****** Running generation 49 ****** 

Population's average fitness: 0.24440 stdev: 0.11549
Best fitness: 0.53551 - size: (11, 28) - species 8 - id 5593
Average adjusted fitness: 0.230
Mean genetic distance 3.258, standard deviation 0.548
Population of 199 members in 20 species
Total extinctions: 0
Generation time: 28.155 sec (27.727 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.25863 stdev: 0.11063
Best fitness: 0.53551 - size: (11, 28) - species 8 - id 5593
Average adjusted fitness: 0.234
Mean genetic distance 3.248, standard deviation 0.554
Population of 195 members in 19 species
Total extinctions: 0
Generation time: 28.596 sec (27.827 average)

 

Population's average fitness: 0.25215 stdev: 0.11804
Best fitness: 0.56392 - size: (12, 31) - species 50 - id 8858
Average adjusted fitness: 0.229
Mean genetic distance 3.178, standard deviation 0.584
Population of 198 members in 12 species
Total extinctions: 0
Generation time: 30.168 sec (31.122 average)

 ****** Running generation 73 ****** 

Population's average fitness: 0.23596 stdev: 0.11432
Best fitness: 0.56392 - size: (12, 31) - species 50 - id 8858
Average adjusted fitness: 0.217
Mean genetic distance 3.196, standard deviation 0.591
Population of 201 members in 12 species
Total extinctions: 0
Generation time: 29.969 sec (31.268 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.25183 stdev: 0.10693
Best fitness: 0.56392 - size: (12, 31) - species 50 - id 8858
Average adjusted fitness: 0.244
Mean genetic distance 3.210, standard deviation 0.599
Population of 198 members in 13 species
Total extinctions: 0
Generation time: 30.407 sec (31.367 average)

Population's average fitness: 0.25192 stdev: 0.11646
Best fitness: 0.56392 - size: (12, 31) - species 50 - id 8858
Average adjusted fitness: 0.234
Mean genetic distance 3.215, standard deviation 0.603
Population of 201 members in 12 species
Total extinctions: 0
Generation time: 30.019 sec (32.237 average)

 ****** Running generation 97 ****** 

Population's average fitness: 0.24081 stdev: 0.11066
Best fitness: 0.56392 - size: (12, 31) - species 50 - id 8858
Average adjusted fitness: 0.142
Mean genetic distance 3.199, standard deviation 0.576
Population of 202 members in 12 species
Total extinctions: 0
Generation time: 30.977 sec (32.292 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.23043 stdev: 0.11137
Best fitness: 0.56392 - size: (12, 31) - species 50 - id 8858
Average adjusted fitness: 0.193
Mean genetic distance 3.197, standard deviation 0.589
Population of 201 members in 13 species
Total extinctions: 0
Generation time: 31.181 sec (31.856 average)

...

Population's average fitness: 0.26108 stdev: 0.11463
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.233
Mean genetic distance 3.004, standard deviation 0.524
Population of 194 members in 16 species
Total extinctions: 0
Generation time: 33.073 sec (33.804 average)

 ****** Running generation 193 ****** 

Population's average fitness: 0.26107 stdev: 0.10693
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.224
Mean genetic distance 3.018, standard deviation 0.511
Population of 196 members in 16 species
Total extinctions: 0
Generation time: 33.951 sec (33.820 average)

 ****** Running generation 194 ****** 

Population's average fitness: 0.26676 stdev: 0.10454
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.190
Mean genetic distance 3.026, standard deviation 0.514
Population of 198 members in 17 species
Total extinctions: 0
Generation time: 34.027 sec (33.882 ave

Population's average fitness: 0.24610 stdev: 0.11382
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.234
Mean genetic distance 2.833, standard deviation 0.502
Population of 202 members in 9 species
Total extinctions: 0
Generation time: 38.159 sec (36.121 average)

 ****** Running generation 217 ****** 

Population's average fitness: 0.24384 stdev: 0.11024
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.227
Mean genetic distance 2.837, standard deviation 0.463
Population of 199 members in 10 species
Total extinctions: 0
Generation time: 37.152 sec (36.448 average)

 ****** Running generation 218 ****** 

Population's average fitness: 0.24714 stdev: 0.11862
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.237
Mean genetic distance 2.795, standard deviation 0.455
Population of 199 members in 9 species
Total extinctions: 0
Generation time: 35.446 sec (36.460 avera

Population's average fitness: 0.25047 stdev: 0.12597
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.214
Mean genetic distance 2.773, standard deviation 0.493
Population of 198 members in 12 species
Total extinctions: 0
Generation time: 34.883 sec (35.903 average)

 ****** Running generation 241 ****** 

Population's average fitness: 0.26687 stdev: 0.11949
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.253
Mean genetic distance 2.741, standard deviation 0.509
Population of 200 members in 12 species
Total extinctions: 0
Generation time: 34.067 sec (35.722 average)

 ****** Running generation 242 ****** 

Population's average fitness: 0.26241 stdev: 0.12010
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.238
Mean genetic distance 2.752, standard deviation 0.504
Population of 204 members in 12 species
Total extinctions: 0
Generation time: 35.146 sec (35.685 ave

Population's average fitness: 0.26158 stdev: 0.11773
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.224
Mean genetic distance 2.730, standard deviation 0.474
Population of 200 members in 11 species
Total extinctions: 0
Generation time: 36.601 sec (35.815 average)

 ****** Running generation 265 ****** 

Population's average fitness: 0.26625 stdev: 0.11591
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.236
Mean genetic distance 2.689, standard deviation 0.484
Population of 200 members in 11 species
Total extinctions: 0
Generation time: 35.611 sec (35.922 average)

 ****** Running generation 266 ****** 

Population's average fitness: 0.24971 stdev: 0.12246
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.229
Mean genetic distance 2.749, standard deviation 0.503
Population of 199 members in 11 species
Total extinctions: 0
Generation time: 35.181 sec (35.888 ave

Population's average fitness: 0.25788 stdev: 0.11533
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.242
Mean genetic distance 2.852, standard deviation 0.540
Population of 198 members in 14 species
Total extinctions: 0
Generation time: 35.381 sec (35.538 average)

 ****** Running generation 289 ****** 

Population's average fitness: 0.26624 stdev: 0.11112
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.244
Mean genetic distance 2.859, standard deviation 0.536
Population of 199 members in 14 species
Total extinctions: 0
Generation time: 34.739 sec (35.548 average)

 ****** Running generation 290 ****** 

Population's average fitness: 0.25560 stdev: 0.11850
Best fitness: 0.57091 - size: (21, 54) - species 55 - id 30819
Average adjusted fitness: 0.240
Mean genetic distance 2.869, standard deviation 0.538
Population of 203 members in 14 species
Total extinctions: 0
Generation time: 45.295 sec (36.762 ave

In [13]:
local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-satellite6")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

In [14]:
test_df = pd.read_csv('sat.tst',header=None,sep = '\s')

testX = test_df.iloc[:,:-1].to_numpy()
testy = test_df.iloc[:,-1].to_numpy()

# 7-> 6 and 1-6 -> 0:5
testy[testy==7] = 6
for i in range(6):
    testy[testy == (i + 1)] = i

In [15]:
winner_net

In [19]:
test_x_inputs =[tuple(c) for c in testX]
test_x_outputs = [tuple([c]) for c in testy]

In [32]:
testsamplesize = 2000
outputs = []
for xi in test_x_inputs:
    output = winner_net.activate(xi)
    outputs.append(output)

px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 6), axis=1)
# the index of maximum in each line
pred_outputs = np.argmax(px_outputs, axis = 1)
real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

acc = np.sum(pred_outputs == real_outputs)/testsamplesize
print("Accuracy: {}".format(acc))

Accuracy: 0.5645
